**Importing Modules and Initiating Dataset**

In [1]:
import pandas as pd

# Reading dataset
df = pd.read_csv(r"C:\Users\antty\Downloads\Github Project\airbnb-project-files\airbnb-recommendation-system-System\airbnb-recommendation-system\raw dataset\AB_NYC_2019.csv")

**Overview Information**

In [20]:
df.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
0,2539,Clean & Quiet Apt Home By The Park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2,355
2,3647,The Village Of HarlemNew York,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,1,365
3,3831,Cozy Entire Floor Of Brownstone,4869,Lisaroxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,1,194
5,5099,Large Cozy 1 Br Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,1,129


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [4]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


**Dropping and Categorzing Columns**

In [5]:
# Drop nonessential columns
df.drop(columns=["reviews_per_month", "last_review"], inplace=True)

In [6]:
# Categorizing appropriate columns for organization
df["room_type"] = df["room_type"].astype("category")
df["neighbourhood_group"] = df["neighbourhood_group"].astype("category")
df["neighbourhood"] = df["neighbourhood"].astype("category")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   id                              48895 non-null  int64   
 1   name                            48879 non-null  object  
 2   host_id                         48895 non-null  int64   
 3   host_name                       48874 non-null  object  
 4   neighbourhood_group             48895 non-null  category
 5   neighbourhood                   48895 non-null  category
 6   latitude                        48895 non-null  float64 
 7   longitude                       48895 non-null  float64 
 8   room_type                       48895 non-null  category
 9   price                           48895 non-null  int64   
 10  minimum_nights                  48895 non-null  int64   
 11  number_of_reviews               48895 non-null  int64   
 12  calculated_host_li

**Handling Missing Values and Duplicates**

In [7]:
df.isnull().sum()

id                                 0
name                              16
host_id                            0
host_name                         21
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                  0
calculated_host_listings_count     0
availability_365                   0
dtype: int64

In [8]:
# Replacing missing names with autogenerated ones, example: "Entire home/apt in Midtown, Manhattan"
df["name"] = df["name"].fillna(df["room_type"].astype(str) + "in" + df["neighbourhood"].astype(str) + ", " + df["neighbourhood_group"].astype(str))

In [9]:
# Adding neutral placeholders for missing host names
df["host_name"] = df["host_name"].fillna("Host Not Disclosed")

In [10]:
# Checking if duplicates in unique booking ids
df = df.drop_duplicates(subset=["id"])

In [11]:
# Checking if theres multiple hosts under same the unique host id
unique_host_names = df.groupby("host_id")["host_name"].nunique()
if (unique_host_names > 1).any():
    print("Multiple hosts")
else:
    print("Unique.")

Unique.


In [12]:
df.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

**Cleaning Host and Listing Names for User Readability**

In [13]:
# Cleaning host names
df["host_name"] = (
    df["host_name"]
    .str.strip()
    .str.title()
    .str.replace(r"[^a-zA-Z0-9 '&-]", '', regex=True)
)

In [14]:
# Cleaning listing names
df["name"] = (
    df["name"]
    .str.strip()
    .str.title()
    .str.replace(r"[^a-zA-Z0-9 '&-]", '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
)

**Setting Boundaries for Prices, Availability and Minimum Nights**

In [15]:
# Removing listings less than $1 and greater than $10,000
df = df[(df["price"] >= 1) & (df["price"] <= 10000)]

In [16]:
# Removing listings with no availability
df = df[(df["availability_365"] >= 1) & (df["availability_365"] <= 365)]

In [17]:
# Removing listings with no minimum nights and minimum nights greater than 365 days
df = df[(df["minimum_nights"] >= 1) & (df["minimum_nights"] <= 365)]

In [18]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
count,3.134000e+04,3.134000e+04,31340.000000,31340.000000,31340.000000,31340.000000,31340.000000,31340.000000,31340.000000
mean,2.100846e+07,8.146908e+07,40.728405,-73.948746,162.102776,8.043395,31.861232,10.339470,175.782993
std,1.148689e+07,8.674522e+07,0.056566,0.051407,254.493457,18.306410,51.646659,40.714127,126.175598
min,2.539000e+03,2.571000e+03,40.499790,-74.244420,10.000000,1.000000,0.000000,1.000000,1.000000
25%,1.167900e+07,8.619906e+06,40.688380,-73.983280,70.000000,2.000000,2.000000,1.000000,55.000000
50%,2.256734e+07,4.109936e+07,40.723725,-73.954115,112.000000,3.000000,10.000000,1.000000,167.000000
75%,3.120184e+07,1.452392e+08,40.763040,-73.930140,189.000000,5.000000,39.000000,3.000000,305.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,365.000000,629.000000,327.000000,365.000000


**Saving Cleaned Dataset**

In [19]:
df.to_csv("Cleaned_Dataset", index=False)